In [ ]:
%load_ext autoreload
%autoreload 2
from aoc.lib import load, timing

YEAR = 2024
DAY = 9

In [ ]:
from dataclasses import dataclass, replace

@dataclass
class Block:
    id: int
    len: int

class Blocklist:
    FREE = -1
    
    def __init__(self, rle):
        self.blocks = []
        for i, (a, b) in enumerate(zip(*(iter(rle),) * 2)):
            self.blocks.append(Block(id=i, len=a))
            self.blocks.append(Block(id=Blocklist.FREE, len=b))
        self.normalise()

    
    def __str__(self):
            return ''.join([''.join(['-' if b.id == Blocklist.FREE else str(b.id)] * b.len) for b in self.blocks])

    
    def normalise(self):
        # First pass: remove empty blocks
        i = 0
        while i < len(self.blocks):
            if self.blocks[i].len == 0:
                del self.blocks[i]
            else:
                i += 1

        # Second pass: remove free at the end
        while self.blocks[-1].id == Blocklist.FREE:
            del self.blocks[-1]
                
        # Third pass: merge matching blocks
        i = 1
        while i < len(self.blocks):
            if self.blocks[i].id == self.blocks[i - 1].id:
                self.blocks[i - 1].len += self.blocks[i].len
                del self.blocks[i]
            else:
                i += 1


    def move(self, idx_from, idx_to):
        # move block idx_from to idx_to
        if self.blocks[idx_to].id != Blocklist.FREE:
            raise ParameterError('target index does not describe free memory')
        if self.blocks[idx_from].id == Blocklist.FREE:
            raise ParameterError('source index describes free memory')
        if self.blocks[idx_to].len <= self.blocks[idx_from].len:
            # completely replace target block
            self.blocks[idx_to].id = self.blocks[idx_from].id
            self.blocks[idx_from].len -= self.blocks[idx_to].len
            self.blocks.insert(idx_from, Block(id=Blocklist.FREE, len=self.blocks[idx_to].len))
        else:
            # complete move, partially replace target block
            self.blocks[idx_to].len -= self.blocks[idx_from].len
            copy = Block(id=self.blocks[idx_from].id, len=self.blocks[idx_from].len)
            self.blocks[idx_from].id = Blocklist.FREE
            self.blocks.insert(idx_to, copy)
        self.normalise()


    def checksum(self):
        index = 0
        checksum = 0
        for b in self.blocks:
            if b.id != Blocklist.FREE:
                # index * id + (index+1) * id + ... = [index * n + sum(0...(n-1))] * id
                checksum += (index * b.len + (b.len - 1) * b.len // 2) * b.id  
            index += b.len
        return checksum
            

@timing
def prepare_data():
    data = load(YEAR, DAY)#, test='2333133121414131402')
    ints = [int(i) for i in data['raw'].strip() + '0']
    return Blocklist(ints)

In [ ]:
# Level 1:
@timing
def level1(blist):
    while True:
        from_idx = len(blist.blocks) - 1
        to_idx = next((i for i, b in enumerate(blist.blocks) if b.id == Blocklist.FREE), None)
        if to_idx is not None and to_idx < from_idx:
            blist.move(from_idx, to_idx)
        else:
            break
    return blist.checksum()

blist = prepare_data()
print(level1(blist))

In [ ]:
# Level 2:
@timing
def level2(blist):
    max_id = max([b.id for b in blist.blocks])
    for from_id in range(max_id + 1)[::-1]:
        from_idx = next((i for i, b in enumerate(blist.blocks) if b.id == from_id), None)
        to_idx = next((i for i, b in enumerate(blist.blocks) if b.id == Blocklist.FREE and b.len >= blist.blocks[from_idx].len), None)
        if to_idx is not None and to_idx < from_idx:
            blist.move(from_idx, to_idx)
        else:
            continue
    return blist.checksum()


blist = prepare_data()
print(level2(blist))